# Question answering using fusion retriever architecture
This notebook builds ontop of [Question answering using embeddings-based search](Question_answering_using_embeddings.ipynb) adding two more concepts
- query rewrite
- re-ranking

This changes will impact the `Search` function (the Retrival part of RAG). 

This architecture is called fusion retriever.

## Installation
Install the Azure Open AI SDK using the below command.

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.14"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.14

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.24129.1"

using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.AIUtilities;

## Run this cell, it will prompt you for the apiKey, endPoint, embeddingDeployment, and chatDeployment

In [6]:
var azureOpenAIKey = await Kernel.GetPasswordAsync("Provide your OPEN_AI_KEY");

// Your endpoint should look like the following https://YOUR_OPEN_AI_RESOURCE_NAME.openai.azure.com/
var azureOpenAIEndpoint = await Kernel.GetInputAsync("Provide the OPEN_AI_ENDPOINT");

// Enter the deployment name you chose when you deployed the model.
var embeddingDeployment = await Kernel.GetInputAsync("Provide embedding deployment name");
var chatDeployment = await Kernel.GetInputAsync("Provide chat deployment name");

### Import namesapaces and create an instance of `OpenAiClient` using the `azureOpenAIEndpoint` and the `azureOpenAIKey`

In [7]:
using Azure;
using Azure.AI.OpenAI;

In [8]:
OpenAIClient client = new (new Uri(azureOpenAIEndpoint), new AzureKeyCredential(azureOpenAIKey.GetClearTextPassword()));

## 1. Prepare search data
To save you the time & expense, we've prepared a pre-embedded dataset of a few hundred Wikipedia articles about the 2022 Winter Olympics.
To see how we constructed this dataset, or to modify it yourself, see [Embedding Wikipedia articles for search](Embedding_Wikipedia_articles_for_search.ipynb)

In [9]:
public record PageBlockWithEmbeddings(string PageTitle, string Block, float[] Embedding);

In [10]:
using System.Text.Json;
using System.Text.Json.Serialization;
using System.IO;

var filePath = Path.Combine("..","..","..","Data","wikipedia_embeddings.json");

var olympicsData = JsonSerializer.Deserialize<PageBlockWithEmbeddings[]>(File.ReadAllText(filePath));

In [11]:
olympicsData.Take(4).DisplayTable();

PageTitle,Block,Embedding
2022 Winter Olympics,"The 2022 Winter Olympics, officially called the XXIV Olympic Winter Games () and commonly known as Beijing 2022 (2022), was an international winter multi-sport event held from 4 to 20 February 2022 in Beijing, China, and surrounding areas with competition in selected events beginning 2 February 2022.{{cite web|title=SuperSport|url=https://supersport.com/news/cd6663a2-8236-44d8-a8b9-4fa192190da7/%7B%7B%20url()-%3Ecurrent()%20%7D%7D|access-date=25 February 2022|website=supersport.com|language=ZA|archive-date=25 February 2022|archive-url=https://web.archive.org/web/20220225173447/https://supersport.com/news/cd6663a2-8236-44d8-a8b9-4fa192190da7/%7B%7B%20url()-%3Ecurrent()%20%7D%7D|url-status=live}} It was the 24th edition of the Winter Olympic Games.","[ -0.007000031, -0.025182178, -0.010695381, -0.005645496, -0.026770474, 0.010612124, -0.01401287, -0.00023596284, -0.0060201543, -0.022453895, 0.03729934, 0.0129561415, -0.030408185, -0.023824442, -0.014550841, -0.027718328, 0.016702726, -0.0068463245, 0.0015754872, -0.015831726 ... (1516 more) ]"
2022 Winter Olympics,"Beijing was selected as host city on 31 July 2015 at the 128th IOC Session in Kuala Lumpur, Malaysia, marking its second time hosting the Olympics, and the last of three consecutive Olympics hosted in East Asia following the 2018 Winter Olympics in Pyeongchang County, South Korea, and the 2020 Summer Olympics in Tokyo, Japan. Having previously hosted the 2008 Summer Olympics, Beijing became the first city to have hosted both the Summer and Winter Olympics. The venues for the Games were concentrated around Beijing, its suburb Yanqing District, and Zhangjiakou, with some events (including the ceremonies and curling) repurposing venues originally built for Beijing 2008 (such as Beijing National Stadium and the Beijing National Aquatics Centre).","[ 0.008893254, -0.012699212, -0.007250349, -0.0007829965, -0.020767841, 0.024421562, -0.03750137, -0.0052427067, -0.0037869278, -0.019334264, 0.016835019, 0.010022355, -0.0055186385, -0.0069966186, -0.02300067, -0.008538032, 0.014373833, -0.011024591, -0.0111577995, 0.0054932656 ... (1516 more) ]"
2022 Winter Olympics,"The Games featured a record 109 events across 15 disciplines, with big air freestyle skiing and women's monobob making their Olympic debuts as medal events, as well as several new mixed competitions. A total of 2,871 athletes representing 91 teams competed in the Games, with Haiti and Saudi Arabia making their Winter Olympic debut.","[ -0.009414442, 0.0101670865, -0.0019868554, -0.023944318, -0.0073287217, 0.01984942, -0.017017433, 0.0011855755, -0.00017719848, -0.03722404, 0.017514944, 0.006266727, -0.013177667, -0.025602689, -0.0065027256, -0.001007779, 0.029697588, -0.017132243, -0.01583106, -0.0075710993 ... (1516 more) ]"
2022 Winter Olympics,"Beijing's hosting of the Games was subject to various concerns and controversies including those related to human rights violations in China, such as the Uyghur genocide, which led to calls for a boycott of the games.{{Cite news|last=Reyes|first=Yacob|date=8 December 2021|title=Beijing Olympics: These countries have announced diplomatic boycotts|work=[[Axios (website)|Axios]]|url=https://www.axios.com/diplomatic-boycott-beijing-olympics-list-countries-73e1240f-b925-40bf-ae67-648e774971c8.html|access-date=5 February 2022|archive-date=4 February 2022|archive-url=https://web.archive.org/web/20220204210817/https://www.axios.com/diplomatic-boycott-beijing-olympics-list-countries-73e1240f-b925-40bf-ae67-648e774971c8.html|url-status=live}}{{Cite news|last1=Allen-Ebrahimian|first1=Bethany|last2=Baker|first2=Kendall|date=1 February 2022|title=The IOC stays silent on human rights in China|work=[[Axios (website)|Axios]]|url=https://www.axios.com/winter-olympics-beijing-ioc-silence-human-rights-31ec1273-d894-4a67-993b-4b4156d42d44.html|access-date=5 February 2022|archive-date=5 February 2022|archive-url=https://web.archive.org/web/202

## 2. Query generation

The `GenerateQueries` function is asynchronous and takes two parameters: `originalQuery` which is a string representing the original search query, and `numQueries` which is an integer representing the number of search queries to generate.

Inside the function, several operations are performed:

1. It creates a prompt string using the `originalQuery` and `numQueries` parameters. This prompt is formatted to instruct an AI assistant to generate `numQueries` search queries related to the `originalQuery`.

2. It creates a new instance of the `ChatCompletionsOptions` class, setting various properties to configure the chat completion request. The `Messages` property is set to a list containing two `ChatMessage` objects: one with the role of `System` and a content of "You answer questions about the 2022 Winter Olympics.", and one with the role of `User` and a content of the previously created prompt. The `Temperature` property is set to 0, the `MaxTokens` property is set to 3500, and the `DeploymentName` property is set to `chatDeployment`.

3. It calls the `GetChatCompletionsAsync` method of the `client` object, passing in the `options` object. This method sends a chat completion request to the OpenAI API and returns a response. The `await` keyword is used to asynchronously wait for the method to complete.

4. It retrieves the content of the first choice from the response and splits it into an array of strings using `Environment.NewLine` as the separator. This array represents the generated search queries.

Finally, it returns the array of generated search queries.



In [12]:
public async Task<string[]> GenerateQueries(string originalQuery, int numQueries)
{
    var prompt = $"""
    You are a helpful assistant that generates multiple search queries based on a
    single input query. Generate {numQueries} search queries, one on each line,
    related to the following input query:
    Query: {originalQuery}
    Queries:
    
    """;
    var options= new ChatCompletionsOptions{
        Messages =
            {
                new ChatRequestSystemMessage(@"You answer questions about the 2022 Winter Olympics."),
                new ChatRequestUserMessage(prompt)
            },
        Temperature = 0f,
        MaxTokens = 3500,
        DeploymentName = chatDeployment
    };

    var response = await client.GetChatCompletionsAsync(options);

    var answer = response.Value.Choices.FirstOrDefault()?.Message?.Content;  

    return answer.Split(Environment.NewLine, StringSplitOptions.RemoveEmptyEntries);
}

In [13]:
await GenerateQueries("How many gold medals in total?", 6)

[ 1. Winter Olympics 2022 gold medal count
2. Breakdown of gold medals won at the 2022 Winter Olympics
3. Total number of gold medals awarded at the 2022 Winter Olympics
4. Gold medal tally for each country at the 2022 Winter Olympics
5. Who won the most gold medals at the 2022 Winter Olympics?
6. Historical comparison of gold medal counts at past Winter Olympics ]

 ## 3. Search
    
 
 Now we'll define a search function that:
 - Takes a user query and a dataframe with text & embedding columns
 - Calculates the text embedding of the query using ADA model
 - Generates additional queries usign GPT model
 - Calculates text embeddings for the  new queries
 - Uses distance between each of the new generatedquery embedding and text embeddings to rank the texts
 - Re-ranks the retrived documents using the original query embeddings
 - Returns a list with:
    - The top N texts, ranked by relevance
    - Their corresponding relevance scores

The `SearchAsync` function is used to perform a search given a query and a collection of knowledge. It takes three parameters: `query` which is a string representing the search query, `knowledge` which is a collection of `PageBlockWithEmbeddings` objects representing the knowledge base, and `resultCount` which is an optional parameter that defaults to 5 and represents the number of search results to return.

This is what happens in the function:

1. It retrieves the embedding for the query using the `GetEmbeddingsAsync` method of the `client` object and stores it in `queryEmbedding`.

2. It generates alternative queries using the `GenerateQueries` method and stores them in `generatedQueries`.

3. It retrieves the embeddings for the generated queries using the `GetEmbeddingsAsync` method of the `client` object and stores them in `generatedQueriesEmbeddings`.

4. It initializes an empty list of `RankedText` objects, named `retrievedFacts`.

5. It enters a loop that iterates over the `generatedQueriesEmbeddings`. For each embedding, it scores the knowledge base by similarity to the embedding, filters out items with a score less than 0.8, takes the top 10 items, and adds them to the `retrievedFacts` list.

6. It scores the `retrievedFacts` by similarity to the `queryEmbedding`, filters out items with a score less than 0.8, takes the top `resultCount` items, and transforms them into `SearchResult` objects. This part is usually referred to as re-ranking.

Finally, it returns the resulting collection of `SearchResult` objects.

In [14]:
public record SearchResult(string Text, float Score);
public record RankedText(string Text, float[] Embedding, float Score);
public async Task<IEnumerable<SearchResult>> SearchAsync(string query, IEnumerable<PageBlockWithEmbeddings> knowledge, int resultCount = 5){
    var response = await client.GetEmbeddingsAsync(new EmbeddingsOptions(embeddingDeployment, new [] {query}));
    var queryEmbedding = response.Value.Data[0].Embedding.ToArray();

    var generatedQueries = await GenerateQueries(query, 10);

    var generatedQueriesEmbeddings = await client.GetEmbeddingsAsync(new EmbeddingsOptions(embeddingDeployment, generatedQueries));

    var retirevedFacts = new List<RankedText>();

    for (int i = 0; i < generatedQueriesEmbeddings.Value.Data.Count; i++)
    {
        var generatedQueryEmbedding = generatedQueriesEmbeddings.Value.Data[i].Embedding.ToArray();
        var docs = knowledge
            .Where(k => !k.Block.StartsWith("Category:"))
            .ScoreBySimilarityTo(generatedQueryEmbedding, new CosineSimilarityComparer<float[]>(t => t),e => e.Embedding.ToArray())
            .OrderByDescending(s => s.Score)
            .Where(s => s.Score > 0.8)
            .Take(20)
            .Select(r => new  RankedText(r.Value.Block, r.Value.Embedding, r.Score));

        retirevedFacts.AddRange(docs);
    }
    var result = retirevedFacts
        .DistinctBy(r => r.Text)
        .ScoreBySimilarityTo(queryEmbedding, new CosineSimilarityComparer<float[]>(t => t),e => e.Embedding.ToArray())
        .OrderByDescending(s => s.Score)
        .Where(s => s.Score > 0.8)
        .Take(resultCount)
        .Select(r => new  SearchResult(r.Value.Text, r.Score));

        return result;  
}

In [15]:
var search = await SearchAsync("curling gold medal", olympicsData);

search.DisplayTable();

Text,Score
"Biathletes Johannes Thingnes Bø, Quentin Fillon Maillet, and Marte Olsbu Røiseland, and cross-country skier Alexander Bolshunov won the most total medals at the games with five each.{{cite web |title=Beijing 2022 |url=https://www.teamgb.com/competitions/beijing-2022/6dWdXrzU85Vn1jF6ZC9Onl |publisher=[[British Olympic Association]] |access-date=26 February 2022 |archive-date=18 March 2022 |archive-url=https://web.archive.org/web/20220318145104/https://www.teamgb.com/competitions/beijing-2022/6dWdXrzU85Vn1jF6ZC9Onl |url-status=live }} Bø also earned the most gold medals with four.{{cite news |author=[[Agence France-Presse]] |title=Norwegian Biathlete Boe Gets Fourth Beijing Olympics Gold Medal |url=https://www.barrons.com/news/norwegian-biathlete-boe-gets-fourth-beijing-olympics-gold-medal-01645189808 |access-date=27 March 2022 |work=[[Barron's (newspaper)|Barron's]] |date=18 February 2022 |archive-date=22 February 2023 |archive-url=https://web.archive.org/web/20230222200651/https://www.barrons.com/news/norwegian-biathlete-boe-gets-fourth-beijing-olympics-gold-medal-01645189808 |url-status=live }} Snowboarder Zoi Sadowski-Synnott of New Zealand won the first Winter Olympic gold medal for that nation.{{cite news |first1=Bryan Armen |last1=Graham |title=Zoi Sadowski-Synnott Wins New Zealand's First Ever Winter Olympic Gold |url=https://www.theguardian.com/sport/2022/feb/05/zoi-sadowski-synnott-new-zealand-first-winter-olympic-gold-snowboard-beijing-2022-tess-coady |access-date=12 July 2022 |work=[[The Guardian]] |date=5 February 2022 |archive-date=26 February 2022 |archive-url=https://web.archive.org/web/20220226175310/https://www.theguardian.com/sport/2022/feb/05/zoi-sadowski-synnott-new-zealand-first-winter-olympic-gold-snowboard-beijing-2022-tess-coady |url-status=live }} Germany achieved a podium sweep in the men's two-man bobsleigh competition with Francesco Friedrich and Thorsten Margis winning gold, Johannes Lochner and Florian Bauer earning silver, and Christoph Hafer and Matthias Sommer attaining bronze.{{cite news |last1=Levinsohn |first1=Dan |title=Germany Sweeps Two-Man Bobsled Podium with Friedrich, Lochner, Hafer |url=https://www.nbcolympics.com/news/recap-two-man-final-heats |access-date=19 February 2022 |agency=[[NBC Sports]] |date=15 February 2022 |archive-date=19 March 2023 |archive-url=https://web.archive.org/web/20230319113128/https://www.nbcolympics.com/news/recap-two-man-final-heats |url-status=live }}",0.83861834
"thumb|Medals of 2022 Winter Olympics Norway finished at the top of the medal table for the second successive Winter Olympics, winning a total of 37 medals, of which 16 were gold, setting a new record for the largest number of gold medals won at a single Winter Olympics. Germany finished second with 12 golds and 27 medals overall, and the host nation China finished third with nine gold medals, marking their most successful performance in Winter Olympics history. The team representing the ROC ended up with the second largest number of medals won at the Games, with 32, but finished ninth on the medal table, as only six gold medals were won by the delegation. Traditional Winter powerhouse Canada; despite having won 26 medals, only four of them were gold, resulting in a finish outside the top ten in the medal table for the first time since 1988 (34 years).{{cite news|first=Spencer|last=Donna|url=https://www.cbc.ca/sports/olympics/winter/beijing-2022-team-canada-wrap-1.6358707|title=Canada caps COVID Olympic Winter Games in Beijing with 26 medals, including 4 gold|agency=[[Canadian Press]]|date=20 February 2022|website=www.cbc.ca/|publisher=[[CBC Sports]]|access-date=22 February 2022|archive-date=19 March 2023|archive-url=https://web.archive.org/web/20230319113124/https://www.cbc.ca/sports/olympics/winter/beijing-2022-team-canada-wrap-1.6358707|url-status=live}}{{cite news|date=20 February 2022|title=Canada finish outside medals top 10 for first time in 34 years|url=https://en.as.com/en/2022/

## 3.Ask
With the search function above, we can now automatically retrieve relevant knowledge and insert it into messages to GPT.

Below, we define a function `AskAsync` that:

 - Takes a user query
 - Searches for text relevant to the query
 - Stuffs that text into a message for GPT
 - Sends the message to GPT
 - Returns GPT's answer

The `AskAsync` method starts by calling the `SearchAsync` method with the user's question and a dataset about the 2022 Winter Olympics (`olympicsData`). The `SearchAsync` method searches the dataset for relevant information and returns a list of search results.

Next, the method constructs a string `articles` that contains all the search results. Each search result is formatted as a section of a Wikipedia article. The search results are joined together with newline characters in between.

The method then constructs a `userQuestion` string that instructs the AI to use the articles to answer the question. If the answer cannot be found in the articles, the AI is instructed to respond with "I could not find an answer."

The `userQuestion` string is then used to create an instance of `ChatCompletionsOptions`. This object is used to specify the parameters for a chat completion request to the OpenAI API. The `Messages` property of the object is set to a list that contains a system message and a user message. The system message instructs the AI that it answers questions about the 2022 Winter Olympics. The user message is the `userQuestion` string. The `Temperature` property is set to 0, which means that the AI will generate more deterministic responses. The `MaxTokens` property is set to 3500, which limits the length of the AI's response. The `DeploymentName` property is set to `chatDeployment`, which likely specifies the deployment of the chat model.

The method then makes an asynchronous request to the OpenAI API to get chat completions. The `GetChatCompletionsAsync` method of the `client` object is used to make this request. The method takes the `ChatCompletionsOptions` instance as a parameter.

Finally, the method processes the response from the OpenAI API to extract the AI's answer. The `Value.Choices.FirstOrDefault()?.Message?.Content` expression is used to get the content of the first choice in the response. The method then returns this answer.

In [16]:
var tokenizer = await Tokenizer.CreateAsync(TokenizerModel.gpt35);

public async Task<string> AskAsync(string question){

    var searchResults = await SearchAsync(question, olympicsData);

    var articles = string.Join("\n", searchResults.Select(s => $"""
    Wikipedia article section:
    {s.Text}

    """));

    var userQuestion = $"""""
                Use the below articles on the 2022 Winter Olympics to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."
                
                
                {articles}
                

                Question: {question}
                """"";

    var options= new ChatCompletionsOptions{
        Messages =
            {
                new ChatRequestSystemMessage(@"You answer questions about the 2022 Winter Olympics."),
                new ChatRequestUserMessage(userQuestion)
            },
        Temperature = 0f,
        MaxTokens = 3500,
        DeploymentName = chatDeployment
    };

    var response = await client.GetChatCompletionsAsync(options);

    var answer = response.Value.Choices.FirstOrDefault()?.Message?.Content;  
    return answer;
}

In [21]:
await AskAsync("How many gold medals in total?")

According to the articles, Norway won a total of 16 gold medals at the 2022 Winter Olympics.

In [22]:
await AskAsync("Where did the 2022 winter Olympics took place?")

The 2022 Winter Olympics took place in Beijing, China.